In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies_df = pd.read_csv('../datasets/movies_dataset.csv', low_memory=False)

Observamos una apariencia general de nuestros datos

In [ ]:
movies_df

<h2>TRANFORMACIONES</h2>

1. Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.
2. Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.
3. Los valores nulos del campo release date deben eliminarse.
4. De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.
5. Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.
6. Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

<H3>APLICANDO TRANSFORMACIONES</H3>

6. Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [ ]:
movies_df = movies_df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']).copy()

1. Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [ ]:

#* Diccionario
display(movies_df.loc[movies_df.belongs_to_collection.index[0]][0])
#* Listas de diccionarios
display(movies_df.loc[movies_df.belongs_to_collection.index[0]][2])

Encontramos la particularidad que en las columnas anidadas hay datos de tipo string con formato de diccionario y strings con formatos de tipo lista con diccionarios dentro

In [ ]:
movies_df.head()

Columnas con datos anidados:
- belongs_to_collection [Dict]
- genres [List]
- production_companies [List]
- production_countries [List]
- spoken_languages [List]


Para evitar problemas con la conversion de datos de tipo str a tipo dict o problemas con las listas y el tipo de dato. Se ha decidido utilizar expresiones regulares para encontrar la informacion relevante dentro de estos registros y solo mostrar estos

In [ ]:
import re

def parse_data(df, column):
    """
    Extrae los valores de una cadena, lista o diccionario en una columna específica de un DataFrame.
    
    Args:
        df (DataFrame): DataFrame en el que se encuentra la columna a procesar.
        column (str): Nombre de la columna a procesar.
    
    Returns:
        DataFrame: DataFrame con la columna modificada, donde se han extraído los valores.
    """
    def extract_values(value):
        """
        Extrae los valores de una cadena, lista o diccionario.
        
        Args:
            value: Valor a procesar.
        
        Returns:
            list or str or None: Valores extraídos.
        """
        if isinstance(value, str):
            # Si es una cadena, extraer los valores con la expresión regular
            pattern = r"'name': '([^']*)'"
            coincidencias = re.findall(pattern, value)
            return coincidencias
        elif isinstance(value, list):
            # Si es una lista, extraer los valores de cada elemento de la lista
            values = []
            for i in value:
                if 'name' in i:
                    values.append(i['name'])
            return values
        elif isinstance(value, dict):
            # Si es un diccionario, extraer los valores de la clave 'name'
            if 'name' in value:
                return value['name']
        return None
    
    # Aplicar la función a la columna y sobrescribir la columna
    df[column] = df[column].apply(extract_values)
    
    return df


In [ ]:
parse_data(movies_df, 'production_countries')
parse_data(movies_df, 'production_companies')
parse_data(movies_df, 'belongs_to_collection')
parse_data(movies_df, 'genres')
parse_data(movies_df, 'spoken_languages')

In [ ]:
movies_df.columns

2.Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [ ]:
movies_df[['revenue', 'budget']] = movies_df[['revenue', 'budget']].fillna(0)   

In [ ]:
movies_df[['revenue', 'budget']].isna().sum()

3. Eliminar registros con fechas nulas

In [ ]:
movies_df.release_date.isna().sum()

Eliminamos los 87 valores nulos como son pedidos

In [ ]:
movies_df.dropna(subset=['release_date'], inplace=True)

Volvemos a comprobar

In [ ]:
movies_df['release_date'].isna().sum()

4. De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [ ]:
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], format='%Y-%m-%d', errors='coerce')
movies_df['release_year'] = movies_df['release_date'].dt.year.fillna(0).astype(int)

5. Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [ ]:
movies_df.dtypes

Como podemos observar en la columna anterior; el dtype de budget y revenue es object y float respectivamente, lo cual no nos permite hacer el calculo aritmentico solicitado.

In [ ]:
# Convierte las columnas 'budget' y 'revenue' a numéricas para poder crear la columna solicitada
movies_df['budget'] = pd.to_numeric(movies_df['budget'], errors='coerce')
movies_df['revenue'] = pd.to_numeric(movies_df['revenue'], errors='coerce')

#* Se transforma tambien esta columna para ahorrar el trabajo de hacerlo despues
movies_df['popularity'] = pd.to_numeric(movies_df['popularity'], errors='coerce')
#Creamos la nueva columna return
movies_df['return'] = movies_df['revenue'] / movies_df['budget']
movies_df['return'] = movies_df['return'].fillna(0)

In [ ]:
movies_df.info()

Los datasets fueron transformados a formato *Parquet* para reducir su peso y su velocidad de analisis. 

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

credits_df = pd.read_csv('../datasets/credits.csv')

# Especificar la ruta y nombre de archivo para el archivo Parquet de salida
output_movies = 'movie_dataset.parquet'
output_credits = 'credits.parquet'
# Convertir el DataFrame de pandas a una tabla de PyArrow
table_movies = pa.Table.from_pandas(movies_df)
table_credits = pa.Table.from_pandas(credits_df)
# Escribir la tabla en formato Parquet
pq.write_table(table_movies, output_movies)
pq.write_table(table_credits, output_credits)

Transformacion del dataset de creditos.

In [ ]:
df_credits = pd.read_parquet('../datasets/credits.parquet')

In [ ]:
import re


def extract_credits_data(df, column):
    def extract_values(value):
        if isinstance(value, str):
            # Si es una cadena, extraer los valores con la expresión regular
            pattern = r"'name': '([^']*)'"
            match = re.findall(pattern, value)
            return match
        elif isinstance(value, list):
            # Si es una lista, extraer los valores de cada elemento de la lista
            values = []
            for i in value:
                if 'name' in i:
                    values.append(i['name'])
            return values
        elif isinstance(value, dict):
            # Si es un diccionario, extraer los valores de la clave 'name'
            if 'name' in value:
                return value['name']
        return None
    
    # Aplicar la función a la columna y sobrescribir la columna
    df[column] = df[column].apply(extract_values)
    
    return df

In [ ]:
extract_credits_data(df_credits, 'cast')
extract_credits_data(df_credits, 'crew')

In [10]:
df_credits.head()

,cast,crew,id
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[John Lasseter, Joss Whedon, Andrew Stanton, J...",862
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Larry J. Franco, Jonathan Hensleigh, James Ho...",8844
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Howard Deutch, Mark Steven Johnson, Mark Stev...",15602
3,"[Whitney Houston, Angela Bassett, Loretta Devi...","[Forest Whitaker, Ronald Bass, Ronald Bass, Ez...",31357
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[Alan Silvestri, Elliot Davis, Nancy Meyers, N...",11862


Transformar el dataframe devuelta a parquet

In [11]:
df_credits.to_parquet('../datasets/credits.parquet', index=False)